In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats
import scipy as sp
from sklearn import metrics
from bayesian_player_ratings import *

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

Load Match Summary Data

In [2]:
match_summary = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/match_summary.csv")
match_summary = score_col_splitter(match_summary, "Q4_Score")
match_summary['Season'] = match_summary['Match_ID'].apply(lambda x: int(x[:4]))

player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats = create_additional_player_stats_variables(player_stats)
player_stats['exp_vaep_value'] = player_stats['exp_vaep_value'].fillna(0)
player_stats['exp_offensive_value'] = player_stats['exp_offensive_value'].fillna(0)
player_stats['exp_defensive_value'] = player_stats['exp_defensive_value'].fillna(0)

match_stats = aggregate_player_to_match_stats(player_stats)

match_summary_stats = match_summary.merge(match_stats, how = "left", on = "Match_ID")

In [3]:
def calculate_probabilities(rating_diff, std):
    
    diff = stats.norm(loc = rating_diff, scale = std)
    
    away_prob = diff.cdf(0)
    draw_prob = diff.pdf(0)
    home_prob = 1 - away_prob - draw_prob
    
    return home_prob, draw_prob, away_prob

In [4]:
def get_round_predictions(player_stats, round_id, match_std = 35):
    
    round_match_list = list(player_stats[player_stats['Round_ID'] == round_id]['Match_ID'].unique())
    
    match_projections_dict = {}
    for match_id in round_match_list:
        match_projections_dict[match_id] = {}
        home_team, away_team = get_teams(match_id)
        home_rating = ratings.team_values[round_id][ratings.team_values[round_id]['Team'] == home_team]['Rating'].iloc[0]
        away_rating = ratings.team_values[round_id][ratings.team_values[round_id]['Team'] != home_team]['Rating'].iloc[0]
        rating_difference = home_rating - away_rating
        home_prob, draw_prob, away_prob = calculate_probabilities(rating_difference, match_std)
        match_projections_dict[match_id]['projected_diff'] = rating_difference
        # match_projections_dict[match_id]['projected_prob'] = home_prob
        
        home_exp_vaep = player_stats[(player_stats['Match_ID'] == match_id) & (player_stats['Team'] == player_stats['Home_Team'])]['exp_vaep_value'].sum()
        away_exp_vaep = player_stats[(player_stats['Match_ID'] == match_id) & (player_stats['Team'] != player_stats['Home_Team'])]['exp_vaep_value'].sum()
        exp_vaep_result = home_exp_vaep - away_exp_vaep
        # match_projections_dict[match_id]['exp_vaep_diff'] = exp_vaep_result        
        
        home_score = match_summary_stats[(match_summary_stats['Match_ID'] == match_id)]['Home_Score'].sum()
        away_score = match_summary_stats[(match_summary_stats['Match_ID'] == match_id)]['Away_Score'].sum()
        result = home_score - away_score
        match_projections_dict[match_id]['margin'] = result
        
        match_projections_dict[match_id]['mae'] = abs(match_projections_dict[match_id]['margin'] - match_projections_dict[match_id]['projected_diff'])

    return match_projections_dict


In [5]:
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
match_summary_stats_2021 = match_summary_stats[match_summary_stats['Season'] == 2021]

In [6]:
ratings = TeamRatings(prior_weight = 1.8)
ratings.calculate_home_advantage(match_summary_stats_2021)
ratings.initialise_player_values(player_stats_2021)
ratings.initialise_team_values(player_stats_2021)
ratings.calculate_home_advantage(match_summary_stats_2021)
ratings.latest_round

'Start'

Rounds

In [7]:
round_projections_dict = {}
round_mae_dict = {}

In [8]:
round_id = '202101'
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.56
Overall MAE: 20.56


,Team,Rating,Offensive_Value,Defensive_Value
13,Richmond,3.7800,16.4028,12.6228
10,Melbourne,2.9955,11.5552,8.5596
12,Port Adelaide,2.9159,13.9961,11.0802
16,West Coast,2.6936,13.1792,10.4856
17,Western Bulldogs,2.6845,12.6532,9.9687
15,Sydney,2.0509,12.1079,10.0570
8,Greater Western Sydney,1.5619,12.7203,11.1583
4,Essendon,0.0582,11.8252,11.7671
0,Adelaide,0.0000,0.0000,0.0000
6,Geelong,0.0000,0.0000,0.0000


In [9]:
round_id = "202102"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)


Round MAE: 24.79
Overall MAE: 22.68


,Team,Rating,Offensive_Value,Defensive_Value
13,Richmond,7.1660,26.8451,19.6791
10,Melbourne,6.8535,23.2040,16.3506
12,Port Adelaide,4.9996,23.4268,18.4272
7,Gold Coast,4.8195,24.1936,19.3741
17,Western Bulldogs,4.1502,24.4634,20.3133
15,Sydney,1.5267,24.5323,23.0056
5,Fremantle,1.1579,21.7474,20.5895
16,West Coast,0.6901,23.0670,22.3769
0,Adelaide,0.3190,14.8608,14.5418
6,Geelong,0.0528,10.4631,10.4103


In [10]:
round_id = "202103"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)


Round MAE: 39.54
Overall MAE: 28.30


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,14.5076,39.9833,25.4757
10,Melbourne,9.5385,33.0343,23.4958
15,Sydney,4.5370,33.4976,28.9606
13,Richmond,3.2865,32.9158,29.6293
16,West Coast,2.9629,32.4268,29.4639
7,Gold Coast,2.5671,31.4635,28.8964
12,Port Adelaide,2.1579,31.3158,29.1579
0,Adelaide,2.0576,26.6974,24.6398
2,Carlton,1.2047,32.0140,30.8093
4,Essendon,0.8983,26.3902,25.4919


In [11]:
round_id = "202104"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)


Round MAE: 13.24
Overall MAE: 24.53


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,16.3998,48.0479,31.6481
10,Melbourne,11.3160,42.4526,31.1366
15,Sydney,5.9000,43.0523,37.1522
2,Carlton,5.7847,41.3431,35.5583
12,Port Adelaide,4.8923,41.0387,36.1464
0,Adelaide,4.4360,36.7341,32.2981
13,Richmond,0.0077,35.0060,34.9984
5,Fremantle,-0.2438,33.7590,34.0028
4,Essendon,-1.0083,34.0919,35.1002
16,West Coast,-1.7005,36.2478,37.9483


In [12]:
round_id = "202105"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 35.73
Overall MAE: 26.77


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,20.1289,54.2971,34.1682
10,Melbourne,15.9894,46.7911,30.8017
13,Richmond,7.0126,48.0677,41.0551
2,Carlton,5.4636,45.7618,40.2982
12,Port Adelaide,4.1457,42.6194,38.4737
15,Sydney,3.6510,46.6541,43.0031
0,Adelaide,2.6464,43.5699,40.9236
1,Brisbane Lions,1.9556,44.5032,42.5476
6,Geelong,1.9513,34.7913,32.8400
5,Fremantle,1.1267,40.9850,39.8583


In [13]:
round_id = "202106"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 37.22
Overall MAE: 28.51


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,22.5617,62.1502,39.5886
10,Melbourne,14.6678,53.3832,38.7154
6,Geelong,7.7920,46.2787,38.4867
12,Port Adelaide,6.6417,50.9341,44.2924
5,Fremantle,6.1016,48.7652,42.6636
13,Richmond,6.0340,52.8359,46.8019
2,Carlton,3.9481,53.4095,49.4614
1,Brisbane Lions,2.7291,54.0125,51.2834
0,Adelaide,1.8406,51.3888,49.5482
7,Gold Coast,-0.9832,49.7617,50.7449


In [14]:
round_id = "202107"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.36
Overall MAE: 29.06


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,18.1337,60.6340,42.5003
10,Melbourne,14.9637,58.4725,43.5088
6,Geelong,10.8355,50.8063,39.9707
13,Richmond,7.6024,55.5254,47.9229
1,Brisbane Lions,6.0239,57.1935,51.1696
2,Carlton,5.0864,59.7383,54.6520
12,Port Adelaide,2.4098,50.6843,48.2745
5,Fremantle,1.0798,50.4094,49.3296
7,Gold Coast,-0.8918,55.3943,56.2861
8,Greater Western Sydney,-2.0918,54.0516,56.1434


In [15]:
round_id = "202108"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.88
Overall MAE: 28.04


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,18.5354,65.9331,47.3977
6,Geelong,15.1304,53.9415,38.8111
10,Melbourne,12.7050,61.9137,49.2087
1,Brisbane Lions,8.1048,65.3326,57.2278
12,Port Adelaide,7.7528,62.7390,54.9862
2,Carlton,2.3626,67.1514,64.7888
13,Richmond,1.4638,58.8253,57.3615
16,West Coast,1.0136,55.7088,54.6952
8,Greater Western Sydney,-0.4588,60.4909,60.9497
5,Fremantle,-1.7114,54.0209,55.7323


In [16]:
round_id = "202109"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.34
Overall MAE: 27.29


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,18.1666,70.4020,52.2354
10,Melbourne,13.5028,64.6785,51.1757
1,Brisbane Lions,13.4678,72.5093,59.0415
6,Geelong,12.9927,58.4805,45.4878
12,Port Adelaide,5.4928,62.7672,57.2744
16,West Coast,2.0553,64.6466,62.5912
13,Richmond,0.5770,53.4557,52.8786
8,Greater Western Sydney,0.3274,57.7188,57.3914
2,Carlton,0.0580,65.8065,65.7485
5,Fremantle,-0.6723,60.1117,60.7840


In [17]:
round_id = "202110"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.91
Overall MAE: 26.96


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,26.1992,79.9369,53.7377
1,Brisbane Lions,15.2611,72.6740,57.4129
10,Melbourne,13.1866,69.4723,56.2857
6,Geelong,12.5271,65.0863,52.5592
12,Port Adelaide,4.2752,60.4892,56.2140
8,Greater Western Sydney,3.3701,59.9384,56.5683
5,Fremantle,1.9096,63.0471,61.1375
2,Carlton,0.7185,70.2049,69.4864
4,Essendon,-0.9872,67.3749,68.3621
16,West Coast,-1.2256,67.8151,69.0408


In [18]:
round_id = "202111"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 27.28
Overall MAE: 26.99


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,21.1442,78.3267,57.1825
1,Brisbane Lions,18.4092,78.9298,60.5206
10,Melbourne,14.3030,78.1327,63.8297
6,Geelong,12.1761,63.7289,51.5528
12,Port Adelaide,5.7756,69.1737,63.3980
4,Essendon,4.7871,72.2929,67.5058
2,Carlton,-0.0911,73.7262,73.8174
5,Fremantle,-0.2093,66.2543,66.4636
8,Greater Western Sydney,-1.6411,59.6191,61.2601
13,Richmond,-1.7642,59.4735,61.2377


In [19]:
round_id = "202112"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.62
Overall MAE: 26.45


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,23.9707,85.8149,61.8441
10,Melbourne,15.8228,81.5539,65.7311
1,Brisbane Lions,13.6182,86.5811,72.9629
6,Geelong,12.1761,63.7289,51.5528
12,Port Adelaide,5.7756,69.1737,63.3980
4,Essendon,3.4283,75.9058,72.4775
2,Carlton,0.3415,75.8243,75.4829
13,Richmond,-0.7077,69.5861,70.2938
15,Sydney,-1.3544,77.4177,78.7721
8,Greater Western Sydney,-1.6411,59.6191,61.2601


In [20]:
round_id = "202113"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.30
Overall MAE: 26.14


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,23.9707,85.8149,61.8441
1,Brisbane Lions,13.6182,86.5811,72.9629
10,Melbourne,13.4911,82.2228,68.7317
6,Geelong,11.6363,72.7945,61.1582
12,Port Adelaide,4.5203,69.5999,65.0797
4,Essendon,3.4283,75.9058,72.4775
13,Richmond,1.1027,72.0664,70.9637
2,Carlton,0.3415,75.8243,75.4829
8,Greater Western Sydney,-0.3478,71.4300,71.7778
0,Adelaide,-2.8133,77.6876,80.5009


In [21]:
round_id = "202114"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 18.58
Overall MAE: 25.60


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,19.9656,86.4648,66.4992
1,Brisbane Lions,16.5303,82.3464,65.8161
10,Melbourne,13.4911,82.2228,68.7317
6,Geelong,12.0808,78.6473,66.5666
12,Port Adelaide,9.1245,73.2236,64.0991
4,Essendon,2.9804,75.3415,72.3611
13,Richmond,1.1027,72.0664,70.9637
8,Greater Western Sydney,0.0037,73.3141,73.3104
2,Carlton,-0.0094,75.7411,75.7505
0,Adelaide,-2.8133,77.6876,80.5009


In [22]:
round_id = "202115"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.24
Overall MAE: 25.31


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,25.2655,88.7611,63.4956
1,Brisbane Lions,18.5416,92.0271,73.4855
10,Melbourne,13.4337,86.9456,73.5119
12,Port Adelaide,9.7136,71.3385,61.6248
6,Geelong,7.2084,78.6813,71.4729
2,Carlton,1.9319,79.6964,77.7645
4,Essendon,1.3907,80.3760,78.9853
13,Richmond,-0.8405,69.9895,70.8299
8,Greater Western Sydney,-2.0378,73.4809,75.5187
3,Collingwood,-3.5360,69.5943,73.1303


In [23]:
round_id = "202116"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.14
Overall MAE: 25.36


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,24.1405,94.0469,69.9064
1,Brisbane Lions,18.3444,91.1086,72.7643
10,Melbourne,12.2504,88.6319,76.3815
12,Port Adelaide,11.1870,76.6514,65.4644
6,Geelong,9.7508,82.6865,72.9357
2,Carlton,1.2968,77.1988,75.9020
15,Sydney,0.9990,87.8195,86.8205
8,Greater Western Sydney,-1.9941,80.4944,82.4885
4,Essendon,-2.0117,79.9211,81.9327
5,Fremantle,-3.2645,75.6048,78.8693


In [24]:
round_id = "202117"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 37.79
Overall MAE: 26.09


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,21.3829,87.7521,66.3692
10,Melbourne,15.4069,95.4444,80.0376
1,Brisbane Lions,14.9935,97.5407,82.5472
6,Geelong,8.8948,78.1637,69.2689
12,Port Adelaide,5.6868,81.0669,75.3801
4,Essendon,4.8646,83.2195,78.3550
5,Fremantle,1.6754,78.4639,76.7885
15,Sydney,1.2426,88.9069,87.6643
2,Carlton,1.0480,82.8975,81.8495
3,Collingwood,-0.2130,73.9631,74.1761


In [25]:
round_id = "202118"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.04
Overall MAE: 26.36


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,17.1145,90.3119,73.1974
10,Melbourne,15.9206,94.7956,78.8749
6,Geelong,14.5279,84.1702,69.6423
1,Brisbane Lions,13.3482,93.6255,80.2773
12,Port Adelaide,6.6913,78.7479,72.0566
4,Essendon,5.4634,82.4407,76.9773
2,Carlton,4.8514,85.0002,80.1488
15,Sydney,0.6387,85.2212,84.5824
8,Greater Western Sydney,-0.4707,75.4740,75.9447
7,Gold Coast,-3.1444,77.1170,80.2614


In [26]:
round_id = "202119"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 27.40
Overall MAE: 26.42


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,17.4917,98.6029,81.1112
17,Western Bulldogs,15.2918,94.9266,79.6348
6,Geelong,14.5284,90.2826,75.7542
10,Melbourne,14.4398,97.9714,83.5315
12,Port Adelaide,8.7558,86.4289,77.6731
4,Essendon,3.8185,83.5309,79.7123
2,Carlton,3.7553,85.5813,81.8260
15,Sydney,3.7527,93.1277,89.3750
8,Greater Western Sydney,0.6748,74.4477,73.7728
14,St Kilda,-4.7213,76.1664,80.8877


In [27]:
round_id = "202120"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.88
Overall MAE: 26.69


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,21.7875,100.8839,79.0964
17,Western Bulldogs,17.4497,97.5463,80.0966
6,Geelong,16.9683,85.4919,68.5236
1,Brisbane Lions,15.8527,100.6053,84.7526
12,Port Adelaide,9.2740,93.4772,84.2033
15,Sydney,3.4516,97.4840,94.0324
4,Essendon,3.3625,85.3290,81.9665
2,Carlton,2.7884,79.6701,76.8817
8,Greater Western Sydney,-0.7864,83.7853,84.5717
3,Collingwood,-2.3925,75.9994,78.3919


In [28]:
round_id = "202121"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.80
Overall MAE: 26.94


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,22.5778,95.0115,72.4336
1,Brisbane Lions,20.1738,98.2954,78.1216
17,Western Bulldogs,19.7351,101.0725,81.3374
6,Geelong,15.5232,86.7924,71.2692
12,Port Adelaide,8.9484,97.3198,88.3714
2,Carlton,1.3139,82.2089,80.8950
15,Sydney,0.0730,98.7638,98.6908
14,St Kilda,-0.2586,83.7243,83.9828
8,Greater Western Sydney,-0.9595,63.0992,64.0587
4,Essendon,-1.0041,82.1124,83.1165


In [29]:
round_id = "202122"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.15
Overall MAE: 27.04


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,23.4640,92.6219,69.1579
1,Brisbane Lions,23.1582,106.7123,83.5541
6,Geelong,16.2359,88.5287,72.2928
17,Western Bulldogs,15.5789,103.0737,87.4948
12,Port Adelaide,15.0173,102.4025,87.3852
4,Essendon,4.6252,88.5782,83.9530
15,Sydney,2.0329,88.8333,86.8004
8,Greater Western Sydney,1.8885,76.6256,74.7370
14,St Kilda,-2.4977,82.0180,84.5157
13,Richmond,-4.4021,76.3673,80.7695


In [30]:
round_id = "202123"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.62
Overall MAE: 27.28


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,23.0691,107.4187,84.3497
10,Melbourne,21.3584,100.0528,78.6944
12,Port Adelaide,15.5148,99.8980,84.3832
6,Geelong,14.3715,91.8873,77.5158
17,Western Bulldogs,12.0218,97.5558,85.5340
4,Essendon,7.5545,94.3127,86.7582
15,Sydney,7.0956,96.3899,89.2943
8,Greater Western Sydney,5.5129,90.7073,85.1944
14,St Kilda,0.1585,76.0682,75.9097
13,Richmond,-2.7452,75.9802,78.7254


In [31]:
round_id = "2021F1"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 16.64
Overall MAE: 26.84


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,21.6618,104.0771,82.4153
1,Brisbane Lions,18.3229,114.5212,96.1983
12,Port Adelaide,17.4463,105.2667,87.8204
17,Western Bulldogs,12.4576,98.2192,85.7616
15,Sydney,9.6287,98.0750,88.4463
6,Geelong,9.4514,92.6983,83.2469
4,Essendon,5.1611,90.1042,84.9431
8,Greater Western Sydney,1.7190,94.6584,92.9394
14,St Kilda,0.1585,76.0682,75.9097
13,Richmond,-2.7452,75.9802,78.7254


In [32]:
round_id = "2021F2"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.87
Overall MAE: 26.60


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,21.6618,104.0771,82.4153
1,Brisbane Lions,17.5140,113.7089,96.1949
12,Port Adelaide,17.4463,105.2667,87.8204
6,Geelong,11.7727,99.7992,88.0265
17,Western Bulldogs,10.1885,98.9199,88.7314
15,Sydney,9.6287,98.0750,88.4463
4,Essendon,5.1611,90.1042,84.9431
14,St Kilda,0.1585,76.0682,75.9097
8,Greater Western Sydney,-1.7193,83.3293,85.0487
13,Richmond,-2.7452,75.9802,78.7254


In [33]:
round_id = "2021F3"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 69.74
Overall MAE: 28.26


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,26.1575,109.3407,83.1832
1,Brisbane Lions,17.5140,113.7089,96.1949
17,Western Bulldogs,13.2393,99.2505,86.0112
12,Port Adelaide,11.6320,103.0586,91.4267
15,Sydney,9.6287,98.0750,88.4463
4,Essendon,5.1611,90.1042,84.9431
6,Geelong,3.9336,94.8919,90.9584
14,St Kilda,0.1585,76.0682,75.9097
8,Greater Western Sydney,-1.7193,83.3293,85.0487
13,Richmond,-2.7452,75.9802,78.7254


In [34]:
round_id = "2021F4"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 36.62
Overall MAE: 28.57


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,28.6386,111.4298,82.7912
1,Brisbane Lions,17.5140,113.7089,96.1949
12,Port Adelaide,11.6320,103.0586,91.4267
15,Sydney,9.6287,98.0750,88.4463
17,Western Bulldogs,6.8185,100.9074,94.0889
4,Essendon,5.1611,90.1042,84.9431
6,Geelong,3.9336,94.8919,90.9584
14,St Kilda,0.1585,76.0682,75.9097
8,Greater Western Sydney,-1.7193,83.3293,85.0487
13,Richmond,-2.7452,75.9802,78.7254


In [35]:
round_id = "202201"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.49
Overall MAE: 28.38


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,30.3106,105.9295,75.6189
1,Brisbane Lions,17.8934,105.7313,87.8379
15,Sydney,9.0072,89.8318,80.8246
12,Port Adelaide,8.3379,94.4720,86.1341
6,Geelong,8.1468,92.2683,84.1214
17,Western Bulldogs,1.6008,98.7319,97.1311
4,Essendon,0.0383,83.9708,83.9324
14,St Kilda,-0.5199,73.6147,74.1346
8,Greater Western Sydney,-1.8888,91.9223,93.8111
13,Richmond,-5.0350,86.5405,91.5755


In [36]:
round_id = "202202"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.34
Overall MAE: 28.38


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,27.0326,103.4823,76.4497
1,Brisbane Lions,16.1073,101.6000,85.4927
15,Sydney,8.2427,93.0144,84.7716
6,Geelong,7.1959,94.5399,87.3439
12,Port Adelaide,6.4319,84.3841,77.9522
17,Western Bulldogs,3.1558,96.1197,92.9639
14,St Kilda,1.1583,74.2813,73.1230
4,Essendon,0.0313,93.9570,93.9258
13,Richmond,-2.0562,75.2084,77.2646
3,Collingwood,-3.8468,87.3278,91.1747


In [37]:
round_id = "202203"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 25.97
Overall MAE: 28.30


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,28.1727,101.0113,72.8386
1,Brisbane Lions,23.9606,111.3466,87.3860
17,Western Bulldogs,7.0307,101.5696,94.5388
12,Port Adelaide,6.6452,84.5040,77.8588
6,Geelong,5.1009,96.4333,91.3324
14,St Kilda,4.9491,82.2410,77.2919
15,Sydney,3.2279,91.8156,88.5877
8,Greater Western Sydney,-1.4292,86.1452,87.5744
3,Collingwood,-2.0867,93.5240,95.6107
4,Essendon,-3.8152,84.5912,88.4064


In [38]:
round_id = "202204"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.03
Overall MAE: 28.23


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,28.2902,104.8672,76.5770
1,Brisbane Lions,19.1670,105.0715,85.9045
14,St Kilda,8.1244,86.9705,78.8461
6,Geelong,6.9883,89.9247,82.9364
17,Western Bulldogs,5.8008,98.1400,92.3392
15,Sydney,3.3774,97.6299,94.2525
12,Port Adelaide,3.0459,86.4213,83.3754
3,Collingwood,1.6359,86.3938,84.7579
5,Fremantle,-2.1614,85.2355,87.3968
4,Essendon,-4.5920,78.2234,82.8155


In [39]:
round_id = "202205"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.20
Overall MAE: 28.26


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,29.1533,109.0962,79.9429
1,Brisbane Lions,19.0269,108.8268,89.8000
17,Western Bulldogs,11.3363,108.9177,97.5814
15,Sydney,8.8890,95.2094,86.3205
14,St Kilda,8.6534,84.7302,76.0768
6,Geelong,6.9020,101.1973,94.2952
12,Port Adelaide,1.6922,87.3223,85.6301
5,Fremantle,0.9674,88.8212,87.8537
3,Collingwood,-0.3043,86.6668,86.9711
13,Richmond,-3.0824,93.2135,96.2958


In [40]:
round_id = "202206"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.64
Overall MAE: 28.30


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,32.1172,107.7984,75.6812
1,Brisbane Lions,18.4544,109.5222,91.0678
15,Sydney,13.0462,104.0163,90.9701
6,Geelong,11.7675,98.2033,86.4358
14,St Kilda,9.3202,87.8926,78.5724
17,Western Bulldogs,6.7902,97.2641,90.4739
12,Port Adelaide,6.6504,98.7362,92.0858
5,Fremantle,4.1051,94.6435,90.5384
3,Collingwood,-1.5399,94.0432,95.5830
0,Adelaide,-4.2760,85.3351,89.6111


In [41]:
round_id = "202207"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.90
Overall MAE: 28.44


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,29.4962,106.3112,76.8150
1,Brisbane Lions,17.6338,112.4577,94.8239
15,Sydney,10.7167,110.9277,100.2109
6,Geelong,9.2943,96.3568,87.0625
14,St Kilda,9.2492,90.9431,81.6940
17,Western Bulldogs,8.8793,101.1874,92.3081
12,Port Adelaide,5.1244,100.9782,95.8538
5,Fremantle,4.9911,89.3976,84.4065
13,Richmond,2.9024,89.7478,86.8454
3,Collingwood,1.8061,93.9599,92.1538


In [42]:
round_id = "202208"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.50
Overall MAE: 28.44


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,28.7645,114.4668,85.7023
1,Brisbane Lions,21.2675,105.9661,84.6986
6,Geelong,12.1015,85.7910,73.6895
5,Fremantle,11.2567,82.9534,71.6967
15,Sydney,7.7338,104.8910,97.1573
12,Port Adelaide,7.2426,103.8223,96.5798
14,St Kilda,6.1064,98.3588,92.2524
17,Western Bulldogs,5.3607,95.6775,90.3168
2,Carlton,3.7471,102.2908,98.5437
13,Richmond,3.3869,97.9853,94.5984


In [43]:
round_id = "202209"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.41
Overall MAE: 28.49


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,31.8142,115.6059,83.7916
1,Brisbane Lions,19.8033,103.6061,83.8028
12,Port Adelaide,12.2546,103.9225,91.6679
15,Sydney,11.6326,110.4485,98.8159
5,Fremantle,9.9639,92.2136,82.2497
17,Western Bulldogs,9.3738,100.4853,91.1115
14,St Kilda,8.9165,103.2981,94.3815
6,Geelong,7.4705,97.4539,89.9834
2,Carlton,7.2830,100.9218,93.6387
13,Richmond,6.6219,106.5607,99.9388


In [44]:
round_id = "202210"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 17.81
Overall MAE: 28.20


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,32.9832,112.5572,79.5740
1,Brisbane Lions,16.6802,104.3898,87.7096
17,Western Bulldogs,12.1491,103.9760,91.8269
13,Richmond,11.3320,102.4817,91.1497
6,Geelong,9.7708,99.0604,89.2895
2,Carlton,8.7618,102.0779,93.3160
15,Sydney,8.2622,110.5847,102.3225
12,Port Adelaide,7.9818,101.6035,93.6217
5,Fremantle,5.9994,100.9525,94.9531
14,St Kilda,5.5116,91.7901,86.2784


In [45]:
round_id = "202211"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.29
Overall MAE: 28.31


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,27.7430,108.2027,80.4597
17,Western Bulldogs,19.6951,112.3330,92.6379
1,Brisbane Lions,18.8079,105.1392,86.3312
6,Geelong,10.6517,90.1043,79.4526
13,Richmond,9.4253,90.8781,81.4528
15,Sydney,8.2095,102.7070,94.4974
14,St Kilda,8.1853,89.4371,81.2519
12,Port Adelaide,7.4877,106.3107,98.8231
5,Fremantle,7.3412,102.0785,94.7373
2,Carlton,4.6278,101.7688,97.1410


In [46]:
round_id = "202212"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.48
Overall MAE: 28.39


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,23.4404,106.6967,83.2563
17,Western Bulldogs,16.9261,118.5375,101.6115
1,Brisbane Lions,16.4240,109.7333,93.3094
6,Geelong,10.3861,97.8355,87.4494
13,Richmond,9.4253,90.8781,81.4528
15,Sydney,8.9169,105.6423,96.7254
14,St Kilda,8.1853,89.4371,81.2519
12,Port Adelaide,7.4877,106.3107,98.8231
5,Fremantle,7.1103,98.3286,91.2183
7,Gold Coast,5.9377,106.7969,100.8592


In [47]:
round_id = "202213"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 8.34
Overall MAE: 27.89


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,17.8121,115.4854,97.6733
10,Melbourne,17.3349,103.2747,85.9398
17,Western Bulldogs,16.9261,118.5375,101.6115
13,Richmond,12.4447,102.6088,90.1641
6,Geelong,10.3861,97.8355,87.4494
15,Sydney,8.9169,105.6423,96.7254
3,Collingwood,7.1817,110.0869,102.9052
5,Fremantle,7.1702,108.0840,100.9137
7,Gold Coast,5.9377,106.7969,100.8592
2,Carlton,5.2707,108.8441,103.5734


In [48]:
round_id = "202214"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.30
Overall MAE: 27.71


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,19.0172,118.5229,99.5057
1,Brisbane Lions,17.8121,115.4854,97.6733
10,Melbourne,17.3349,103.2747,85.9398
6,Geelong,12.3672,98.6606,86.2934
13,Richmond,11.7933,107.7350,95.9416
7,Gold Coast,8.2474,110.5744,102.3270
3,Collingwood,7.1817,110.0869,102.9052
5,Fremantle,7.1702,108.0840,100.9137
15,Sydney,6.2658,109.3272,103.0614
12,Port Adelaide,4.2587,98.2239,93.9652


In [49]:
round_id = "202215"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.57
Overall MAE: 27.58


,Team,Rating,Offensive_Value,Defensive_Value
17,Western Bulldogs,21.0760,112.0944,91.0184
10,Melbourne,20.9141,108.0199,87.1057
6,Geelong,11.4804,106.3161,94.8356
1,Brisbane Lions,10.7181,101.3602,90.6421
13,Richmond,10.2640,103.3132,93.0493
3,Collingwood,10.0289,104.6271,94.5982
15,Sydney,9.7007,102.1947,92.4940
7,Gold Coast,7.8236,114.6527,106.8291
2,Carlton,5.2433,107.0958,101.8525
5,Fremantle,4.9454,106.7970,101.8515


In [50]:
round_id = "202216"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 24.41
Overall MAE: 27.51


,Team,Rating,Offensive_Value,Defensive_Value
10,Melbourne,22.8162,102.6706,79.8544
6,Geelong,20.2050,105.5680,85.3631
17,Western Bulldogs,16.8965,105.7256,88.8291
1,Brisbane Lions,11.7182,113.2168,101.4986
3,Collingwood,10.2220,109.4227,99.2008
15,Sydney,9.9798,107.1454,97.1656
13,Richmond,9.7754,103.6955,93.9202
2,Carlton,6.4572,111.2744,104.8172
5,Fremantle,5.8524,112.2124,106.3599
7,Gold Coast,5.8453,104.7617,98.9165


In [51]:
round_id = "202217"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.71
Overall MAE: 27.49


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.0637,112.8067,89.7430
10,Melbourne,15.6553,111.1042,95.4489
17,Western Bulldogs,12.6629,113.7948,101.1320
2,Carlton,11.9825,113.7296,101.7471
15,Sydney,11.5259,106.9885,95.4627
13,Richmond,11.4716,103.8588,92.3873
3,Collingwood,10.8489,103.2699,92.4210
1,Brisbane Lions,8.5790,89.4654,80.8864
5,Fremantle,7.9601,116.5565,108.5964
12,Port Adelaide,3.7365,105.9117,102.1752


In [52]:
round_id = "202218"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 27.00
Overall MAE: 27.48


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,24.0970,112.0789,87.9819
10,Melbourne,13.8156,103.5933,89.7777
13,Richmond,12.8049,102.6752,89.8704
15,Sydney,12.0652,107.4084,95.3432
1,Brisbane Lions,11.6432,93.8536,82.2104
17,Western Bulldogs,10.8882,110.9503,100.0621
3,Collingwood,9.6931,98.7815,89.0884
2,Carlton,7.4446,106.9036,99.4590
5,Fremantle,5.4721,115.7169,110.2447
4,Essendon,3.7062,102.0152,98.3090


In [53]:
round_id = "202219"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 13.99
Overall MAE: 27.19


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,21.3622,105.4381,84.0759
15,Sydney,14.9582,112.3883,97.4301
10,Melbourne,12.0826,104.7155,92.6329
13,Richmond,11.9748,102.3414,90.3666
1,Brisbane Lions,10.7805,108.6177,97.8372
2,Carlton,10.3652,110.6386,100.2734
17,Western Bulldogs,10.1508,119.1670,109.0162
3,Collingwood,7.4384,99.9223,92.4838
4,Essendon,4.6209,100.4364,95.8155
5,Fremantle,4.4745,111.3822,106.9077


In [54]:
round_id = "202220"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.03
Overall MAE: 27.10


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,21.3562,108.5725,87.2163
15,Sydney,20.5225,116.1051,95.5826
10,Melbourne,14.2133,109.0717,94.8584
13,Richmond,10.4632,109.9728,99.5096
1,Brisbane Lions,10.0166,106.9408,96.9242
2,Carlton,7.0994,111.2658,104.1665
3,Collingwood,7.0154,102.2495,95.2341
17,Western Bulldogs,7.0055,113.7491,106.7437
4,Essendon,5.3292,96.6148,91.2856
12,Port Adelaide,2.9177,104.5064,101.5886


In [55]:
round_id = "202221"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 21.34
Overall MAE: 26.98


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.8450,109.1489,85.3039
15,Sydney,21.2653,117.7470,96.4817
10,Melbourne,14.5328,117.9536,103.4208
13,Richmond,13.1190,103.4180,90.2990
1,Brisbane Lions,11.2530,113.8930,102.6401
17,Western Bulldogs,7.2008,112.8333,105.6325
3,Collingwood,4.5730,102.8140,98.2409
2,Carlton,4.1514,102.9557,98.8043
4,Essendon,1.6669,103.5928,101.9259
5,Fremantle,-0.2766,104.8480,105.1246


In [56]:
round_id = "202222"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.22
Overall MAE: 27.07


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,27.2649,112.0064,84.7416
15,Sydney,22.2597,118.4328,96.1731
13,Richmond,15.5407,105.2803,89.7395
10,Melbourne,13.1732,116.7600,103.5868
1,Brisbane Lions,11.7945,114.1181,102.3235
17,Western Bulldogs,7.7431,112.8185,105.0754
12,Port Adelaide,4.2336,105.8620,101.6283
5,Fremantle,3.8791,106.5984,102.7192
2,Carlton,3.6426,97.2961,93.6535
3,Collingwood,0.9948,94.0765,93.0817


In [57]:
round_id = "202223"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 37.36
Overall MAE: 27.27


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,33.3035,108.1210,74.8174
15,Sydney,22.2927,117.8505,95.5578
13,Richmond,18.7289,108.0913,89.3624
10,Melbourne,14.1902,117.2201,103.0299
17,Western Bulldogs,12.3022,113.6626,101.3604
12,Port Adelaide,10.1047,110.6547,100.5499
1,Brisbane Lions,8.2807,110.6199,102.3391
5,Fremantle,6.5113,108.4514,101.9401
2,Carlton,5.3015,100.6681,95.3666
3,Collingwood,-1.1279,102.1236,103.2514


In [58]:
round_id = "2022F1"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.18
Overall MAE: 27.25


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,31.1067,114.0688,82.9621
15,Sydney,20.7846,117.0724,96.2879
13,Richmond,15.8102,115.0416,99.2314
10,Melbourne,12.0501,116.5258,104.4758
17,Western Bulldogs,11.3301,104.6380,93.3079
12,Port Adelaide,10.1047,110.6547,100.5499
1,Brisbane Lions,8.4985,110.6578,102.1593
5,Fremantle,5.6020,107.3785,101.7765
2,Carlton,5.3015,100.6681,95.3666
3,Collingwood,-2.1486,102.9059,105.0545


In [59]:
round_id = "2022F2"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.75
Overall MAE: 27.16


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,31.1067,114.0688,82.9621
15,Sydney,20.7846,117.0724,96.2879
13,Richmond,15.8102,115.0416,99.2314
10,Melbourne,12.6547,117.0647,104.4100
17,Western Bulldogs,11.3301,104.6380,93.3079
12,Port Adelaide,10.1047,110.6547,100.5499
1,Brisbane Lions,5.8389,92.8390,87.0001
2,Carlton,5.3015,100.6681,95.3666
5,Fremantle,2.9616,104.9391,101.9775
3,Collingwood,0.1465,99.3286,99.1821


In [60]:
round_id = "2022F3"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 29.00
Overall MAE: 27.20


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,31.6504,115.2741,83.6237
15,Sydney,19.6482,118.3964,98.7483
13,Richmond,15.8102,115.0416,99.2314
10,Melbourne,12.6547,117.0647,104.4100
17,Western Bulldogs,11.3301,104.6380,93.3079
12,Port Adelaide,10.1047,110.6547,100.5499
2,Carlton,5.3015,100.6681,95.3666
5,Fremantle,2.9616,104.9391,101.9775
1,Brisbane Lions,1.6007,101.4871,99.8863
3,Collingwood,-0.8102,100.4050,101.2152


In [61]:
round_id = "2022F4"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 34.83
Overall MAE: 27.34


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,34.5628,116.3467,81.7839
13,Richmond,15.8102,115.0416,99.2314
10,Melbourne,12.6547,117.0647,104.4100
15,Sydney,11.6059,113.7701,102.1642
17,Western Bulldogs,11.3301,104.6380,93.3079
12,Port Adelaide,10.1047,110.6547,100.5499
2,Carlton,5.3015,100.6681,95.3666
5,Fremantle,2.9616,104.9391,101.9775
1,Brisbane Lions,1.6007,101.4871,99.8863
3,Collingwood,-0.8102,100.4050,101.2152


In [62]:
round_id = "202301"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.75
Overall MAE: 27.44


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,28.9687,109.2648,80.2961
12,Port Adelaide,17.8572,106.8777,89.0205
13,Richmond,15.8907,121.1376,105.2470
10,Melbourne,15.1393,103.3197,88.1803
15,Sydney,14.8594,109.9423,95.0829
17,Western Bulldogs,6.4471,109.3888,102.9418
2,Carlton,3.1099,111.2065,108.0966
5,Fremantle,2.3749,107.5271,105.1523
3,Collingwood,1.4086,116.0626,114.6540
4,Essendon,-2.5162,98.9763,101.4925


In [63]:
round_id = "202302"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.47
Overall MAE: 27.51


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,23.9950,100.9646,76.9696
15,Sydney,17.9826,107.2132,89.2306
13,Richmond,14.3569,115.1042,100.7473
12,Port Adelaide,12.1155,105.0740,92.9585
10,Melbourne,11.7726,107.6766,95.9040
3,Collingwood,5.2238,113.3984,108.1746
2,Carlton,4.8757,105.9595,101.0838
5,Fremantle,3.0169,105.0479,102.0310
17,Western Bulldogs,2.7454,102.1077,99.3622
4,Essendon,-1.1429,96.7575,97.9005


In [64]:
round_id = "202303"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 19.25
Overall MAE: 27.37


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,20.3826,104.0978,83.7152
15,Sydney,14.7277,108.3698,93.6421
10,Melbourne,12.0520,103.0739,91.0219
13,Richmond,10.8791,96.3985,85.5193
12,Port Adelaide,8.9475,109.7258,100.7782
3,Collingwood,6.7434,109.0297,102.2862
2,Carlton,6.5701,108.1589,101.5888
5,Fremantle,4.0664,99.9629,95.8965
17,Western Bulldogs,2.8778,104.9458,102.0681
14,St Kilda,-1.1885,73.6009,74.7895


In [65]:
round_id = "202304"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.60
Overall MAE: 27.39


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,24.7825,110.4165,85.6340
10,Melbourne,14.8664,107.1352,92.2688
15,Sydney,14.0634,113.0951,99.0318
13,Richmond,8.4070,100.3871,91.9801
12,Port Adelaide,7.2443,100.8368,93.5925
2,Carlton,6.2769,97.6615,91.3846
3,Collingwood,4.4358,107.0008,102.5650
17,Western Bulldogs,3.9742,107.1514,103.1771
14,St Kilda,2.2981,78.2414,75.9433
5,Fremantle,1.3954,96.3270,94.9316


In [66]:
round_id = "202305"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 22.75
Overall MAE: 27.31


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,26.0550,113.7007,87.6458
15,Sydney,14.0634,113.0951,99.0318
10,Melbourne,10.1608,103.9712,93.8104
12,Port Adelaide,9.0461,104.8830,95.8370
13,Richmond,8.4070,100.3871,91.9801
3,Collingwood,4.4358,107.0008,102.5650
4,Essendon,3.9364,113.6522,109.7159
5,Fremantle,2.5380,101.3061,98.7681
14,St Kilda,2.2981,78.2414,75.9433
0,Adelaide,1.2938,91.9452,90.6514


In [67]:
round_id = "202306"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 27.34
Overall MAE: 27.31


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,29.5045,113.2504,83.7459
12,Port Adelaide,11.6049,98.9722,87.3673
10,Melbourne,8.9731,105.7035,96.7303
13,Richmond,7.7379,103.6809,95.9430
15,Sydney,6.6020,101.5142,94.9122
17,Western Bulldogs,6.5029,113.4430,106.9401
3,Collingwood,5.0015,99.5945,94.5930
4,Essendon,2.5334,99.7619,97.2285
14,St Kilda,2.2251,88.9712,86.7461
0,Adelaide,0.6615,90.7020,90.0405


In [68]:
round_id = "202307"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 31.51
Overall MAE: 27.38


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,29.3451,110.4965,81.1514
10,Melbourne,13.1302,111.0991,97.9690
12,Port Adelaide,10.5714,95.7852,85.2138
2,Carlton,9.5059,113.4056,103.8997
17,Western Bulldogs,8.0723,110.6623,102.5899
15,Sydney,7.5236,104.2024,96.6788
13,Richmond,5.3602,93.8267,88.4665
1,Brisbane Lions,4.3394,97.4463,93.1069
3,Collingwood,2.7249,95.5519,92.8270
0,Adelaide,2.3717,92.7453,90.3735


In [69]:
round_id = "202308"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 20.39
Overall MAE: 27.26


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,28.6778,109.4876,80.8098
12,Port Adelaide,14.0010,103.1017,89.1007
10,Melbourne,11.7659,112.0242,100.2583
13,Richmond,7.9575,107.6785,99.7210
17,Western Bulldogs,7.2646,115.5344,108.2698
2,Carlton,6.7152,108.8856,102.1704
1,Brisbane Lions,5.7455,94.2645,88.5190
15,Sydney,5.5654,107.8188,102.2534
3,Collingwood,3.6583,104.9051,101.2468
14,St Kilda,3.0291,94.0129,90.9838


In [70]:
round_id = "202309"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 33.82
Overall MAE: 27.37


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,26.4677,98.2643,71.7966
12,Port Adelaide,18.1866,97.8654,79.6787
10,Melbourne,14.5599,113.5295,98.9697
1,Brisbane Lions,9.6800,95.5821,85.9020
17,Western Bulldogs,6.9768,108.5461,101.5693
7,Gold Coast,6.7190,93.1515,86.4326
13,Richmond,6.5040,112.6595,106.1555
3,Collingwood,6.0823,100.9757,94.8935
2,Carlton,5.6050,110.1976,104.5926
0,Adelaide,3.2772,87.5868,84.3096


In [71]:
round_id = "202310"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 32.72
Overall MAE: 27.45


,Team,Rating,Offensive_Value,Defensive_Value
6,Geelong,22.2965,96.2206,73.9241
12,Port Adelaide,18.7226,102.7424,84.0197
1,Brisbane Lions,12.8302,100.6496,87.8194
17,Western Bulldogs,11.1048,110.9950,99.8902
10,Melbourne,10.7492,110.0544,99.3052
3,Collingwood,7.6005,108.0266,100.4262
13,Richmond,5.6790,108.8913,103.2123
15,Sydney,3.1557,99.1712,96.0156
5,Fremantle,2.9193,105.8171,102.8978
2,Carlton,2.9181,107.6058,104.6877


In [72]:
round_id = "202311"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 18.12
Overall MAE: 27.31


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,19.5431,100.3349,80.7918
6,Geelong,18.5869,97.0350,78.4481
1,Brisbane Lions,12.0516,102.7621,90.7106
10,Melbourne,11.4512,103.6684,92.2172
17,Western Bulldogs,10.8424,113.1566,102.3141
3,Collingwood,8.0930,107.5835,99.4904
15,Sydney,2.9277,101.3470,98.4193
2,Carlton,2.5387,103.0873,100.5486
13,Richmond,2.4184,104.4387,102.0203
7,Gold Coast,0.8879,98.7220,97.8341


In [73]:
round_id = "202312"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 16.55
Overall MAE: 27.15


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,20.6907,106.3458,85.6551
6,Geelong,15.8852,97.3394,81.4541
10,Melbourne,14.1817,102.2023,88.0206
3,Collingwood,12.8923,106.6233,93.7310
1,Brisbane Lions,12.0516,102.7621,90.7106
17,Western Bulldogs,10.6011,114.7832,104.1821
13,Richmond,3.6649,102.6375,98.9726
15,Sydney,2.9277,101.3470,98.4193
7,Gold Coast,2.1786,100.2884,98.1098
5,Fremantle,0.8504,99.0415,98.1911


In [74]:
round_id = "202313"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.58
Overall MAE: 27.20


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,21.1005,114.5519,93.4514
6,Geelong,15.8852,97.3394,81.4541
10,Melbourne,15.5104,103.5834,88.0730
1,Brisbane Lions,9.8089,98.2201,88.4112
3,Collingwood,8.8562,96.8170,87.9608
17,Western Bulldogs,7.0622,110.9374,103.8751
0,Adelaide,6.1111,96.2084,90.0972
13,Richmond,3.1351,107.3277,104.1926
14,St Kilda,2.9039,88.4749,85.5711
7,Gold Coast,2.1786,100.2884,98.1098


In [75]:
round_id = "202314"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 34.10
Overall MAE: 27.30


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,22.9076,119.1092,96.2017
10,Melbourne,15.5104,103.5834,88.0730
1,Brisbane Lions,12.7820,83.7717,70.9897
6,Geelong,10.3795,112.7622,102.3826
17,Western Bulldogs,8.9937,118.7818,109.7881
3,Collingwood,8.8562,96.8170,87.9608
0,Adelaide,6.1111,96.2084,90.0972
2,Carlton,4.6249,102.4001,97.7752
13,Richmond,4.1006,105.6876,101.5869
14,St Kilda,1.3344,87.5465,86.2121


In [76]:
round_id = "202315"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 51.23
Overall MAE: 27.65


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,22.9076,119.1092,96.2017
10,Melbourne,13.1913,106.5853,93.3940
1,Brisbane Lions,12.9354,86.8230,73.8876
15,Sydney,10.5495,112.0931,101.5436
6,Geelong,10.1097,108.4372,98.3275
17,Western Bulldogs,8.9937,118.7818,109.7881
3,Collingwood,8.2938,101.1554,92.8616
0,Adelaide,5.1767,98.3973,93.2206
2,Carlton,4.6249,102.4001,97.7752
13,Richmond,4.1006,105.6876,101.5869


In [77]:
round_id = "202316"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 39.12
Overall MAE: 27.81


,Team,Rating,Offensive_Value,Defensive_Value
12,Port Adelaide,19.1381,117.8700,98.7320
1,Brisbane Lions,18.0323,90.8846,72.8524
10,Melbourne,15.6012,112.2260,96.6248
3,Collingwood,13.9246,108.0284,94.1039
15,Sydney,12.9011,107.5683,94.6672
0,Adelaide,10.8707,98.7224,87.8517
2,Carlton,8.9368,109.3848,100.4480
17,Western Bulldogs,8.4593,114.4714,106.0121
6,Geelong,5.6922,109.9895,104.2973
14,St Kilda,4.5204,88.2229,83.7025


In [78]:
round_id = "202317"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 30.84
Overall MAE: 27.85


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,24.5082,94.0687,69.5605
12,Port Adelaide,20.7420,114.9995,94.2574
10,Melbourne,12.7426,102.6588,89.9162
3,Collingwood,12.5499,107.4931,94.9432
6,Geelong,12.4083,104.3395,91.9312
2,Carlton,11.5044,110.9103,99.4058
15,Sydney,7.6092,108.2776,100.6685
17,Western Bulldogs,7.5956,113.5400,105.9444
0,Adelaide,6.9483,99.7930,92.8447
14,St Kilda,5.3669,83.5191,78.1522


In [79]:
round_id = "202318"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 24.22
Overall MAE: 27.80


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,23.0632,98.2910,75.2278
6,Geelong,17.8836,110.5107,92.6271
12,Port Adelaide,16.0187,113.7838,97.7651
3,Collingwood,13.0616,110.2654,97.2037
2,Carlton,13.0031,110.0719,97.0688
10,Melbourne,10.4625,96.0251,85.5626
15,Sydney,7.4412,105.6068,98.1656
17,Western Bulldogs,6.2430,107.0589,100.8159
0,Adelaide,5.0128,104.9358,99.9230
13,Richmond,4.5990,103.5113,98.9123


In [80]:
round_id = "202319"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 19.69
Overall MAE: 27.69


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,21.5093,87.3759,65.8666
2,Carlton,18.5177,85.6325,67.1148
12,Port Adelaide,17.3646,116.2175,98.8529
6,Geelong,15.3428,105.2020,89.8591
10,Melbourne,9.7755,95.5580,85.7825
3,Collingwood,8.8077,115.0567,106.2490
15,Sydney,8.5971,119.5640,110.9669
17,Western Bulldogs,8.1664,121.3754,113.2090
13,Richmond,7.3013,101.6576,94.3563
0,Adelaide,4.1522,95.6831,91.5309


In [81]:
round_id = "202320"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 28.56
Overall MAE: 27.70


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,16.8911,87.8882,70.9970
2,Carlton,15.7628,95.2641,79.5012
6,Geelong,13.1901,111.8681,98.6781
10,Melbourne,12.5213,99.4733,86.9520
12,Port Adelaide,11.9631,117.6044,105.6414
3,Collingwood,8.8300,113.6424,104.8123
15,Sydney,8.0981,122.9092,114.8111
0,Adelaide,7.4020,93.9983,86.5963
17,Western Bulldogs,7.1753,115.6563,108.4810
14,St Kilda,6.3474,85.1480,78.8006


In [82]:
round_id = "202321"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 23.33
Overall MAE: 27.64


,Team,Rating,Offensive_Value,Defensive_Value
1,Brisbane Lions,16.0154,90.9249,74.9095
2,Carlton,15.8488,87.4715,71.6227
10,Melbourne,13.2761,105.5098,92.2336
6,Geelong,12.2586,109.0544,96.7959
17,Western Bulldogs,11.8847,115.6034,103.7187
0,Adelaide,11.2962,87.8896,76.5935
12,Port Adelaide,10.3793,89.0590,78.6798
15,Sydney,6.5707,118.0942,111.5235
3,Collingwood,4.7811,108.2346,103.4536
14,St Kilda,4.0504,86.2030,82.1526


In [83]:
round_id = "202322"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 26.59
Overall MAE: 27.63


,Team,Rating,Offensive_Value,Defensive_Value
2,Carlton,13.6833,87.3049,73.6216
10,Melbourne,12.5291,110.2879,97.7588
1,Brisbane Lions,12.4884,96.1421,83.6537
12,Port Adelaide,12.1954,108.4921,96.2966
0,Adelaide,12.0920,86.6894,74.5974
17,Western Bulldogs,9.6428,118.3939,108.7511
6,Geelong,9.3298,103.7648,94.4350
15,Sydney,7.6543,114.3307,106.6764
3,Collingwood,6.0058,114.9743,108.9685
14,St Kilda,5.0425,92.7015,87.6590


In [84]:
round_id = "202323"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: 34.65
Overall MAE: 27.72


,Team,Rating,Offensive_Value,Defensive_Value
2,Carlton,12.5949,97.8698,85.2749
10,Melbourne,12.5291,110.2879,97.7588
12,Port Adelaide,12.1954,108.4921,96.2966
1,Brisbane Lions,11.3704,102.5713,91.2009
0,Adelaide,10.3000,93.2300,82.9300
14,St Kilda,9.5547,94.7783,85.2237
8,Greater Western Sydney,9.2834,111.0708,101.7874
17,Western Bulldogs,8.7621,112.1202,103.3580
15,Sydney,7.4716,118.5591,111.0874
3,Collingwood,5.2744,104.9925,99.7181


In [85]:
round_id = "202324"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: nan
Overall MAE: nan


,Team,Rating,Offensive_Value,Defensive_Value
2,Carlton,12.5949,97.8698,85.2749
10,Melbourne,12.5291,110.2879,97.7588
12,Port Adelaide,12.1954,108.4921,96.2966
1,Brisbane Lions,11.3704,102.5713,91.2009
0,Adelaide,10.3000,93.2300,82.9300
14,St Kilda,9.5547,94.7783,85.2237
8,Greater Western Sydney,9.2834,111.0708,101.7874
17,Western Bulldogs,8.7621,112.1202,103.3580
15,Sydney,7.4716,118.5591,111.0874
3,Collingwood,5.2744,104.9925,99.7181


In [86]:
round_id = "2023F1"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

Round MAE: nan
Overall MAE: nan


,Team,Rating,Offensive_Value,Defensive_Value
2,Carlton,12.5949,97.8698,85.2749
10,Melbourne,12.5291,110.2879,97.7588
12,Port Adelaide,12.1954,108.4921,96.2966
1,Brisbane Lions,11.3704,102.5713,91.2009
0,Adelaide,10.3000,93.2300,82.9300
14,St Kilda,9.5547,94.7783,85.2237
8,Greater Western Sydney,9.2834,111.0708,101.7874
17,Western Bulldogs,8.7621,112.1202,103.3580
15,Sydney,7.4716,118.5591,111.0874
3,Collingwood,5.2744,104.9925,99.7181


In [ ]:
round_id = "2023F2"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

In [ ]:
round_id = "2023F3"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)

In [ ]:
round_id = "2023F4"
ratings.update_round_values(player_stats, round_id)
round_projections_dict[round_id] = get_round_predictions(player_stats, round_id, match_std = 35)
round_mae_dict[round_id] = np.mean(np.array(list({v['mae'] for (k, v) in round_projections_dict[round_id].items()})))
print("Round MAE: {:.2f}".format(round_mae_dict[round_id]))
print("Overall MAE: {:.2f}".format(np.mean(np.array(list(round_mae_dict.values())))))
ratings.team_values[round_id].sort_values(by='Rating', ascending = False)